In [ ]:
# Core Libraries for Data and RAG
%pip install pandas faiss-cpu langchain langchain-community langchain-huggingface

# Libraries for the Local LLM (Hugging Face)
%pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-c

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving student_behavior_log.csv to student_behavior_log.csv


In [ ]:
#Convert data to Langhain documents. Candidate can combine other columns as well such as Subject#

import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

# 1. Load the Dummy Data
df = pd.read_csv("student_behavior_log.csv")

# 2. Pre-processing: Create a combined text field for the Context
# We combine Name, Class, and Notes so the model has full context in one string.
df['combined_text'] = (
    "Student Name: " + df['Student_ID'] +
    "; Class: " + df['Class'].astype(str) +
    "; Notes: " + df['Teacher_Notes']
)

# 3. Convert to LangChain Documents
loader = DataFrameLoader(df, page_content_column="combined_text")
documents = loader.load()

print(f"Loaded {len(documents)} documents. Here is one example:")
print(documents[0].page_content)

FileNotFoundError: [Errno 2] No such file or directory: 'student_behavior_log.csv'

In [ ]:
# The Embedding and indexing Part. This uses a small, fast model (all-MiniLM-L6-v2) and FAISS for vector storage.
# Knowledge Check for candidate: Chunking is not required in this use case because:
# Tabular Data, Notes column doesn't have huge text, each row acts as chunks

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Initialize the Embedding Model (Runs on CPU easily)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Create the Vector Store (This indexes the documents)
# In an interview, ask: "Why does this step take time?" (Ans: Converting text to numbers)
vector_db = FAISS.from_documents(documents, embedding_model)

print("Vector Database created successfully!")

Vector Database created successfully!


In [ ]:
# The LLM & RAG Chain (The "Generation" Engine)
#For the interview, we use google/flan-t5-large.
#It is fast (2GB), runs on CPU, and is smart enough for this specific task without needing a GPU.

from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# 1. Load a Small, Local LLM (Flan-T5)
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create a pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
local_llm = HuggingFacePipeline(pipeline=pipe)

# 2. Build the Retrieval Chain
from langchain_community.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff", # 'Stuff' means putting all retrieved docs into the prompt
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}), # Retrieve top 3 matches
    return_source_documents=True # Helpful to see WHICH docs were used
)

print("RAG Chain is ready to answer questions!")

ModuleNotFoundError: No module named 'langchain_huggingface'